In [ ]:
!pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import time
from itertools import cycle

import torch
from Bio import SeqIO
import numpy as np
from sklearn.preprocessing import label_binarize
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import cohen_kappa_score,confusion_matrix,classification_report, precision_score, recall_score, f1_score, accuracy_score, roc_curve, \
    auc,roc_auc_score
from numpy.lib import interp
import matplotlib.pyplot as plt

#roc
from scipy import interp
from sklearn.preprocessing import label_binarize

print(torch.cuda.device_count())
print(torch.cuda.is_available())

1
True


In [ ]:
import torch.nn.functional as F


def feature(f):
    amino_acids = "XACDEFGHIKLMNPQRSTVWY"
    amino_acids_dict = {}
    seqs = []
    lable_seqs = []

    for n, s in enumerate(amino_acids):
        amino_acids_dict[s] = n

    for n, s in enumerate(SeqIO.parse(f, "fasta")):
        temp_pad = []
        temp_pad1 = []
        temps = []
        for i in range(20):
            temp_pad1.append(0)
        for i in range(100 - len(s)):
            temps.append(temp_pad1)
        for i in range(100 - len(str(s.seq))):
            temp_pad.append(0)
        train_seq = [amino_acids_dict[a.upper()] for a in str(s.seq).upper()] + temp_pad
        seqs.append(train_seq)

        lable_seqs.append([int(s.id[-1])])

    return seqs, lable_seqs


def Load_Data(f, flag):
    seqs, lable_seqs = feature(f)

    seqs = np.array(seqs)
    lable_seqs = np.array(lable_seqs)
    if flag:
        shuffle_idx = np.random.permutation(np.arange(len(seqs)))
        seqs = seqs[shuffle_idx]

        lable_seqs = (torch.from_numpy(lable_seqs[shuffle_idx]).long()) - 1

        temp = torch.zeros(lable_seqs.shape[0], 4)

        lable_seqs = temp.scatter_(1, lable_seqs, 1)

        return torch.from_numpy(seqs), lable_seqs
    else:
        lable_seqs = torch.from_numpy(lable_seqs).long() - 1

        temp = torch.zeros(lable_seqs.shape[0], 4)

        lable_seqs = temp.scatter_(1, lable_seqs, 1)

        return torch.from_numpy(seqs), (lable_seqs)


class BiLSTM_Attention(nn.Module):
    def __init__(self, num_layer):
        super(BiLSTM_Attention, self).__init__()
        self.num_layer = num_layer
        self.embedding = nn.Embedding(21, 128)
        self.lstm = nn.LSTM(128, 64, num_layer, bidirectional=True)
        self.out = nn.Linear(256 * int((self.num_layer) / 2), 4)

    def attention_net(self, lstm_output, final_state):
        hidden = final_state.view(-1, 64 * 2, self.num_layer)

        attn_weights = torch.bmm(lstm_output, hidden)
        attn_weights = attn_weights.view(attn_weights.shape[0], -1)
        soft_attn_weights = F.softmax(attn_weights, 1)
        soft_attn_weights = soft_attn_weights.view(soft_attn_weights.shape[0], -1, self.num_layer)

        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights)
        context = context.view(context.shape[0], -1)
        return context, soft_attn_weights

    def forward(self, X):
        input = self.embedding(X.long().permute(1, 0))

        hidden_state = Variable(torch.zeros(1 * 2 * self.num_layer, len(X), 64)).to(
            "cuda:0" if torch.cuda.is_available() else "cpu")
        cell_state = Variable(torch.zeros(1 * 2 * self.num_layer, len(X), 64)).to(
            "cuda:0" if torch.cuda.is_available() else "cpu")

        output, (final_hidden_state, final_cell_state) = self.lstm(input, (hidden_state, cell_state))
        output = output.permute(1, 0, 2)
        attn_output, attention = self.attention_net(output, final_hidden_state)

        return torch.nn.functional.log_softmax(self.out(attn_output), 1), attention, output


class Melt_Lstm(torch.nn.Module):
    def __init__(self):
        super(Melt_Lstm, self).__init__()

        self.e1 = nn.Embedding(21, 128)

        self.lstm = nn.LSTM(128, 32, num_layers=4, bidirectional=True, dropout=0.1)

        self.lstm_a1 = BiLSTM_Attention(4)

        self.conv = nn.Conv1d(100, 50, 1, 1)

        self.conv1 = nn.Conv1d(50, 25, 3, 1)

        self.maxpool1 = nn.MaxPool1d(1, 1)

        self.maxpool = nn.MaxPool1d(3, 1)

        self.fc1 = nn.Linear(9124, 4)

        self.fc2 = nn.Linear(12500, 6024)

        self.fc3 = nn.Linear(6400, 4)

        self.fc4 = nn.Linear(12400, 4)

    def forward(self, x):
        x1 = x.permute(1, 0)

        x_1_ = self.e1(x1.long())

        xa1 = self.conv(x_1_.permute(1, 0, 2))

        xa1 = self.maxpool(xa1)

        xa1 = self.conv1((xa1))

        xa1_1 = self.maxpool1(xa1)

        xa1 = xa1_1.view(xa1_1.shape[0], -1)

        xa_1 = self.conv(x_1_.permute(1, 0, 2))

        xa_1 = self.maxpool(xa_1)

        xa_1 = self.conv1((xa_1))

        xa_11 = self.maxpool1(xa_1)

        xa_1 = xa_11.view(xa_11.shape[0], -1)

        xx = torch.cat((xa_11, xa1_1, xa_11, xa1_1), 1)
        xx1 = xx.view(xx.shape[0], -1)
        xx1 = torch.nn.functional.log_softmax((self.fc4(xx1)), 1)

        h_2A, _ = self.lstm(x_1_)

        h_2 = h_2A.permute(1, 0, 2)

        h_2 = h_2.reshape(h_2.shape[0], -1)

        h_2 = torch.nn.functional.log_softmax((self.fc3(h_2)), 1)

        h_1, attention1, output = self.lstm_a1(x)

        h13 = torch.cat((output, x_1_.permute(1, 0, 2), xx), 2)

        h11 = self.conv(h13)

        h1 = self.maxpool(h11)

        h1 = self.conv1(h1)

        h1 = self.maxpool1(h1)

        h1 = h1.reshape(h1.shape[0], -1)

        out = torch.nn.functional.relu(self.fc2(torch.cat((h1, xa_1), 1)))

        a = torch.nn.functional.log_softmax(self.fc1(torch.cat((out, xa1), 1)), 1)

        a = (a + h_2 + h_1 + xx1) / 4

        return a


def train(seqs, ind_seqs, lable_train, ind_seqs_lable_train):
    r_data = torch.utils.data.TensorDataset(ind_seqs, ind_seqs_lable_train)

    r_dataloder = DataLoader(r_data, shuffle=False, batch_size=128, drop_last=False)

    torch_data = torch.utils.data.TensorDataset(seqs, lable_train)

    dataloder = DataLoader(torch_data, shuffle=False, batch_size=128, drop_last=False)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    Cnn_model = Melt_Lstm().to(device)

    loss_func = torch.nn.functional.nll_loss

    optimizer = torch.optim.Adam(Cnn_model.parameters(), lr=1e-3)

    for epoch in range(35):
        loss_it = 0
        total = 0
        train_Acc = 0

        Cnn_model.train()
        for seq, lable_seq in dataloder:
            total += len(seq)

            train_d = seq.to(device).type(torch.cuda.FloatTensor)
            lable_seq = lable_seq.to(device).type(torch.cuda.FloatTensor)
            optimizer.zero_grad()
            out = Cnn_model(train_d)

            loss = loss_func(out, torch.argmax(lable_seq, 1))
            loss.backward()
            optimizer.step()

            loss_it += loss.item()

            train_Acc += torch.sum(torch.argmax(out, 1).eq_(torch.argmax(lable_seq, 1))).cpu().tolist()

        score = 0
        print("epoch: {} --> loss: {:5F} --> train_Acc: {:5F} --> val_Acc: {:5F}".format(epoch, (loss_it / total),
                                                                                         (train_Acc / total), score))

    multi_model = Cnn_model.eval()

    r_test_data = 0
    r_test_Acc = 0
    y_pred = []
    true_label = []
    for r_te_seq, r_te_lable_seq in r_dataloder:
        r_te_seq = r_te_seq.to(device).type(torch.cuda.FloatTensor)

        r_te_lable_seq = r_te_lable_seq.to(device).type(torch.cuda.FloatTensor)

        r_test_data += len(r_te_seq)

        r_preds = multi_model(r_te_seq)

        r_preds = torch.softmax(r_preds, 1)

        true_label += torch.argmax(r_te_lable_seq, 1).cpu().tolist()

        y_pred += torch.argmax(r_preds, 1).cpu().tolist()

        r_test_Acc += torch.sum(torch.argmax(r_preds, 1).eq_(torch.argmax(r_te_lable_seq, 1))).cpu().tolist()

    print("---------train------------")

    decode_save_dict = {"Multi_model1": multi_model.state_dict()}
    #
    torch.save(decode_save_dict, "/content/drive/MyDrive/GM-Pep/data/Multi_model_v1.txt")

    classification_report(true_label, y_pred, digits=4)

    return classification_report(true_label, y_pred, digits=4)

In [ ]:
def eval(file_predicted):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"

    amino_acids_dict = {}

    for n, s in enumerate(amino_acids):
        amino_acids_dict[n] = s

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    ind_seqs, ind_seqs_lable_train = Load_Data(file_predicted, True)

    r_data = torch.utils.data.TensorDataset(ind_seqs, ind_seqs_lable_train)

    r_dataloder = DataLoader(r_data, shuffle=False, drop_last=False, batch_size=128)

    Cnn_model = Melt_Lstm().to(device)

    save_dict = torch.load("/content/drive/MyDrive/GM-Pep/data/Multi_model_v1.txt")

    Cnn_model.load_state_dict(save_dict["Multi_model1"], strict=True)

    Cnn_model = Cnn_model.eval()

    r_test_data = 0
    r_test_Acc = 0
    y_pred = []
    true_label = []
    n = 1
    prob_list=[]
    for r_te_seq, r_te_lable_seq in r_dataloder:
        r_te_seq = r_te_seq.to(device).type(torch.cuda.FloatTensor)
        r_te_lable_seq = r_te_lable_seq.to(device).type(torch.cuda.FloatTensor)
        r_test_data += len(r_te_seq)
        r_preds = Cnn_model(r_te_seq)
        true_label += torch.argmax(r_te_lable_seq, 1).cpu().tolist()
        y_pred += torch.argmax(r_preds, 1).cpu().tolist()
        prob = np.round(torch.softmax(r_preds, 1).detach().cpu().numpy(), 4)
        prob_list.extend(prob)
        n += 1
        r_test_Acc += torch.sum(torch.argmax(r_preds, 1).eq_(torch.argmax(r_te_lable_seq, 1))).cpu().tolist()

    print("---------test------------")
    print(y_pred)
    print(len(y_pred))
    print(true_label)
    print(classification_report(true_label, y_pred, digits=4))
    return true_label,prob_list

In [ ]:
def draw_mutiROC(y_test,prob):
    n_classes = 4
    y_score = prob
    print(y_score)

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # Binarize the output 将类别标签二值化
    y_test = label_binarize(y_test, classes=[0, 1, 2, 3])
    y_test=np.array(y_test)
    y_score=np.array(y_score)
    # one vs rest方式计算每个类别的TPR/FPR以及AUC
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    # 微平均方式计算TPR/FPR，最后得到AUC
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    # 直接调用函数计算
    micro_auc = roc_auc_score(y_test, y_score, average='micro')

    lw = 2

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    # 直接调用函数计算
    macro_auc = roc_auc_score(y_test, y_score, average='macro')

    print(roc_auc)
    print('micro auc:', micro_auc)
    print('macro auc:', macro_auc)

    # Plot all ROC curves
    plt.figure(figsize=(19.2, 10.8),dpi=600)
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]),
             color='deeppink', linestyle='solid', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'.format(roc_auc["macro"]),
             color='navy', linestyle='solid', linewidth=4)

    colors = ['aqua', 'darkorange', 'cornflowerblue','red']
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))
    plt.plot(0, 1,label='perfect separation',color='green', marker='o')

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.savefig('save.png')
    plt.show()

In [ ]:

seqs, lable_train = Load_Data("benchmark.txt", True)
ind_seqs, ind_seqs_lable_train = Load_Data("independent.txt", True)
print(seqs.size())
print(lable_train.size())
r = train(seqs, ind_seqs, lable_train, ind_seqs_lable_train)

torch.Size([8017, 100])
torch.Size([8017, 4])
epoch: 0 --> loss: 0.004978 --> train_Acc: 0.856929 --> val_Acc: 0.000000
epoch: 1 --> loss: 0.002461 --> train_Acc: 0.944742 --> val_Acc: 0.000000
epoch: 2 --> loss: 0.001662 --> train_Acc: 0.955220 --> val_Acc: 0.000000
epoch: 3 --> loss: 0.001211 --> train_Acc: 0.965324 --> val_Acc: 0.000000
epoch: 4 --> loss: 0.000933 --> train_Acc: 0.977797 --> val_Acc: 0.000000
epoch: 5 --> loss: 0.000782 --> train_Acc: 0.981539 --> val_Acc: 0.000000
epoch: 6 --> loss: 0.000708 --> train_Acc: 0.985531 --> val_Acc: 0.000000
epoch: 7 --> loss: 0.000618 --> train_Acc: 0.987776 --> val_Acc: 0.000000
epoch: 8 --> loss: 0.000537 --> train_Acc: 0.991269 --> val_Acc: 0.000000
epoch: 9 --> loss: 0.000491 --> train_Acc: 0.992391 --> val_Acc: 0.000000
epoch: 10 --> loss: 0.000422 --> train_Acc: 0.993514 --> val_Acc: 0.000000
epoch: 11 --> loss: 0.000338 --> train_Acc: 0.995135 --> val_Acc: 0.000000
epoch: 12 --> loss: 0.000324 --> train_Acc: 0.995260 --> val_Acc

In [ ]:
y_pred,prob_list= eval("/content/drive/MyDrive/GM-Pep/data/independent.txt")

---------test------------
[3, 3, 3, 0, 0, 2, 3, 3, 3, 3, 3, 2, 2, 3, 3, 2, 0, 3, 2, 3, 3, 0, 3, 3, 2, 3, 2, 1, 1, 0, 0, 0, 3, 3, 3, 3, 1, 2, 3, 3, 3, 0, 0, 0, 3, 2, 1, 3, 3, 1, 1, 2, 3, 0, 3, 3, 3, 3, 3, 1, 2, 3, 1, 2, 3, 2, 1, 3, 3, 0, 3, 3, 0, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 0, 3, 3, 3, 0, 3, 0, 2, 2, 3, 1, 1, 0, 1, 3, 3, 3, 3, 2, 3, 0, 3, 2, 0, 1, 3, 3, 3, 3, 3, 2, 3, 1, 3, 2, 0, 3, 3, 0, 1, 3, 3, 3, 3, 0, 1, 1, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 3, 3, 0, 3, 3, 0, 3, 2, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 2, 3, 2, 3, 1, 3, 3, 0, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 0, 3, 3, 3, 2, 3, 3, 0, 1, 0, 3, 0, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 0, 2, 0, 2, 3, 2, 3, 3, 1, 3, 1, 3, 3, 3, 0, 2, 3, 3, 3, 0, 3, 3, 0, 0, 1, 3, 3, 2, 3, 2, 1, 3, 3, 1, 0, 3, 0, 0, 1, 3, 3, 3, 0, 2, 0, 0, 3, 0, 3, 2, 0, 1, 3, 1, 0, 1, 3, 0, 3, 3, 3, 1, 3, 0, 3, 0, 3, 3, 3, 3, 3, 0, 0, 1, 3, 0, 3, 3, 3, 2, 3, 0, 3, 3, 0, 3, 1, 3, 3, 3, 0, 1, 0, 3, 3, 1, 3, 3, 1, 1, 2

In [ ]:
draw_mutiROC(y_pred,prob_list)

[array([0., 0., 0., 1.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([9.998e-01, 1.000e-04, 0.000e+00, 1.000e-04], dtype=float32), array([1., 0., 0., 0.], dtype=float32), array([0., 0., 1., 0.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([2.400e-03, 8.000e-04, 6.000e-04, 9.963e-01], dtype=float32), array([0.000e+00, 1.110e-02, 9.889e-01, 1.000e-04], dtype=float32), array([0., 0., 1., 0.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([0., 0., 1., 0.], dtype=float32), array([1., 0., 0., 0.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([0., 0., 1., 0.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([1., 0., 0., 0.], dtype=float32), array([0., 0., 0., 1.], dtype=float

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead
